In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from transformers import AutoModel, AutoProcessor
import torch

## Load model

In [ ]:
model = AutoModel.from_pretrained("visheratin/MC-LLaVA-3b", torch_dtype=torch.float16, trust_remote_code=True).to("cuda")
processor = AutoProcessor.from_pretrained("visheratin/MC-LLaVA-3b", trust_remote_code=True)

## Load the image

In [ ]:
raw_image = Image.open('../sample_images/IMG_1694.jpg')

In [ ]:
plt.imshow(raw_image)
plt.show()

In [ ]:
prompt = """<|im_start|>user
<image>
Describe the image.<|im_end|>
<|im_start|>assistant
"""

In [ ]:
with torch.inference_mode():
    inputs = processor(prompt, [raw_image], model, max_crops=100, num_tokens=728)

In [ ]:
from transformers import TextStreamer

streamer = TextStreamer(processor.tokenizer)

In [ ]:
with torch.inference_mode():
  output = model.generate(**inputs, max_new_tokens=200, do_sample=True, use_cache=False, top_p=0.9, temperature=1.2, eos_token_id=processor.tokenizer.eos_token_id, streamer=streamer)

In [ ]:
print(processor.tokenizer.decode(output[0]).replace(prompt, "").replace("<|im_end|>", ""))